In [1]:
from IPython import get_ipython
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  #'last', 'last_expr'

In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F

In [3]:
def foo(x, y):
    a = torch.sin(x)
    b = torch.cos(y)
    return a + b
opt_foo1 = torch.compile(foo)
a, b = torch.randn(10, 10), torch.randn(10, 10)
print(opt_foo1(a.to(0), b.to(0)))

tensor([[ 0.2748,  0.5467,  1.9439,  0.4705,  1.2793, -0.2457,  0.7065, -0.4898,
          1.9260,  1.5952],
        [ 0.1441,  0.8569, -1.2726,  0.3970,  1.1867,  0.1359,  0.3979,  1.0589,
         -1.7756,  0.8207],
        [-0.0428,  0.9587,  1.7211, -0.3067, -0.3189,  0.8541,  0.0526,  1.9741,
          0.5461,  1.0095],
        [ 1.7107,  0.5056, -0.0739, -0.4487, -0.8810,  0.2943,  0.3522,  1.7507,
         -1.0195,  0.5647],
        [ 0.2748,  1.0458,  1.5435,  0.8252,  1.1416,  1.4460,  1.6930, -0.0650,
         -0.4000, -0.0466],
        [ 1.8172,  0.0346,  0.1227,  0.5867,  0.2079,  1.4106,  0.4992, -0.1673,
          1.0171,  1.6174],
        [ 1.2070,  1.1640,  0.6587,  0.8267, -0.2508, -0.1096,  0.4644, -0.0699,
          1.7652,  0.8288],
        [ 0.7397, -0.4255,  1.8645, -0.0685,  0.1234,  0.4751,  0.2722,  1.3570,
          0.4018, -0.2655],
        [ 1.5860, -0.0666,  1.1341,  1.0162, -0.0681,  1.9191,  0.2502,  0.5994,
          1.7928,  1.7523],
        [ 0.7424,  

In [4]:
def rmsnorm(inputs):
    var = np.mean(np.square(inputs), axis=-2, keepdims=True)
    return inputs / np.sqrt(var + 1e-6)

In [5]:
# B, T, D, N, I = 4, 16, 4096, 32, 2; S = T
B, T, D, N, I = 4, 16, 2048, 16, 2; S = T
HD = N * I * 2; print('HD =', HD)
x = np.random.randn(B, T, D)
dw1 = np.random.randn(D, HD * 2) * np.sqrt(2/(D + HD * 1))  # + HD * 2
hidden = x @ dw1; print('hidden:', hidden.shape, hidden.std())

q_hidden, k_hidden = np.split(hidden, 2, axis=-1)
print('q_hidden:', q_hidden.shape, q_hidden.std())

qw = np.random.randn(HD, N, I * 2) * np.sqrt(1 / HD) * 2 / (N + I) * 0.01; print('qw:', qw.shape, qw.std())
qw1, qw2 = np.split(np.einsum('BTK,KNI->BTNI', q_hidden, qw), 2, axis=-1); print('qw1/2:', qw1.shape, qw1.std(), qw2.std())
qw1 = rmsnorm(qw1); print('qw1:', qw1.shape, qw1.std())

HD = 64
hidden: (4, 16, 128) 1.368727324844607
q_hidden: (4, 16, 64) 1.3501020927830765
qw: (64, 16, 4) 0.00013758410245601785
qw1/2: (4, 16, 16, 2) 0.0015094095864997892 0.0014835413300252301
qw1: (4, 16, 16, 2) 0.8199719259850679


In [ ]:
qw1 = np.random.randn(B, T, N, I) * np.sqrt(2 / (N + I))
qw2 = np.random.randn(B, T, N, I) * np.sqrt(2 / (N + I))

In [6]:
inputs = np.random.randn(B, N, T, S); print('inputs:', inputs.shape, inputs.std())
hidden = np.einsum('BNTS,BTNI->BITS', inputs, qw1); print('hidden:', hidden.shape, hidden.std())
qout = np.einsum('BITS,BTNI->BNTS', hidden, qw2); print('qout:', qout.shape, qout.std())

kout = np.random.randn(B, N, T, S) * qout.std()
out = qout + kout; print('out:', out.shape, out.std())

inputs: (4, 16, 16, 16) 0.9980158533354072
hidden: (4, 2, 16, 16) 3.265709170132894
qout: (4, 16, 16, 16) 0.006964823501883651
out: (4, 16, 16, 16) 0.009851364194778629


In [7]:
dd = np.random.randn(D, N * 2) * np.sqrt(2 / (D + N * 1)) * 0.005; print('dd:', dd.shape, dd.std())
qdd, kdd = np.split(np.einsum('BTD,DN->BTN', x, dd), 2, axis=-1); print('q/kdd:', qdd.shape, qdd.std(), kdd.std())

qdout = np.einsum('BNTS,BTN->BNTS', inputs, qdd); print('qdout:', qdout.shape, qdout.std())

kdout = np.random.randn(B, N, T, S) * qdout.std(); print('kdout:', kdout.shape,qdout.std())
out = qdout + kdout; print('out:', out.shape, out.std())

dd: (2048, 32) 0.0001560595921996125
q/kdd: (4, 16, 16) 0.007099319258200077 0.007054335266385006
qdout: (4, 16, 16, 16) 0.006977773284341642
kdout: (4, 16, 16, 16) 0.006977773284341642
out: (4, 16, 16, 16) 0.009884432699261078


In [ ]:
def timed(fn):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()
    result = fn()
    end.record()
    torch.cuda.synchronize()
    return result, start.elapsed_time(end) / 1000

In [ ]:
@torch.jit.script
def attn(x, wq, wk, wv, wo, sw, dw, K, V, KW, KDD):
    B, T, S, N, D, I = 1, 1, 1024, 32, 128, 2; E = D * N
    q, k, v = x @ wq, x @ wk, x @ wv
    q, k, v = [a.view(B, T, N, D).permute((0, 2, 1, 3)) for a in [q, k, v]]  # BTND->BNTD
    K = torch.cat([K, k], dim=2); V = torch.cat([V, v], dim=2)  # BNSD,BN1D->BN(S+1)D
    logits = q @ K.transpose(-2, -1)  # BNTS
    probs = logits.softmax(-1)
    
#     w1, w2, dd = (x @ dw).view(B, T, 2, N, -1).split([I, I, 1], dim=-1)  # BTE,E(2N(2*I+1))->BT(2N(2*I+1))->BT2N(2*I+1)->[BT2NI]*2,BT2N
#     qw, kw = torch.einsum('BTKNI,BTKMI->BTKNM', w1, w2).unbind(dim=2)  # k=2, BT2NM->[BTNM]*2
#     KW = torch.cat([KW, kw], dim=1)  # BSNM,B1NM->B(S+1)NM
#     w = sw + qw + KW  # NM,B1NM,BSNM->BSNM
#     qdd, kdd = dd.squeeze(-1).unbind(dim=2) # BT2N1->BT2N->[BTN]*2
#     KDD = torch.cat([KDD, kdd], dim=1)  # BSN,B1N->B(S+1)N
#     d = qdd + KDD  # B1N,BSN->BSN
#     probs = torch.einsum('BNTS,BSNM->BMTS', probs, w) + torch.einsum('BNTS,BSN->BNTS', probs, d)
    
    w1, w2, dd = (x @ dw).view(B, T, 2, N, -1).split([I, I, 1], dim=-1)  # BTE,E(2N(2*I+1))->BT(2N(2*I+1))->BT2N(2*I+1)->[BT2NI]*2,BT2N1
    w1 = w1.transpose(-2, -1)  # BT2NI->BT2IN
#     w1 = w1 * torch.rsqrt(w1.pow(2).mean(-1, keepdim=True) + 1e-6)
    w1 = F.layer_norm(w1, (N,))
    qw, kw = torch.einsum('BTKIN,BTKMI->BKNMT', w1, w2).unbind(dim=1)  # k=2, B2NMT->[BNMT]*2
    KW = torch.cat([KW, kw], dim=-1)  # BNMS,BNM1->BNM(S+1)
    w = sw.unsqueeze(-1) + qw + KW  # NM1,BNM1,BNMS->BNMS
#     qdd, kdd = dd.squeeze(-1).unbind(dim=2) # BT2N1->BT2N->[BTN]*2
#     KDD = torch.cat([KDD, kdd], dim=1)  # BSN,B1N->B(S+1)N
    qdd, kdd = dd.squeeze(-1).permute((0, 2, 3, 1)).unbind(dim=1) # BT2N1->BT2N->B2NT->[BNT]*2
    KDD = torch.cat([KDD, kdd], dim=-1)  # BNS,BN1->BN(S+1)
    d = qdd + KDD  # B1N,BSN->BSN
    probs = torch.einsum('BNTS,BNMS->BMTS', probs, w) + torch.einsum('BNTS,BNS->BNTS', probs, d)
    
    o = probs @ V  # BNTS,BNSD->BNTD
    o = o.permute((0, 2, 1, 3)).view(B, T, E) @ wo  # BNTD->BTND->BT(ND)
    return K, V, KW, KDD

In [ ]:
@torch.jit.script
def attn(x, wq, wk, wv, wo, sw, dw, K, V, KW, KDD):
    B, T, S, N, D, I = 1, 1, 1024, 32, 128, 2; E = D * N
    q, k, v = x @ wq, x @ wk, x @ wv
    q, k, v = [a.view(B, T, N, D).permute((0, 2, 1, 3)) for a in [q, k, v]]  # BTND->BNTD
    K = torch.cat([K, k], dim=2); V = torch.cat([V, v], dim=2)  # BNSD,BN1D->BN(S+1)D
    logits = q @ K.transpose(-2, -1)  # BNTS
    
    project_logits = True
    shape = (B,T,2,2,N,-1) # if project_logits else (B,T,2,N,-1)
    w1, w2, dd = (x @ dw).view(shape).split([I, I, 1], -1) # BTE,E(2{2}N(2*I+1))->BT(2{2}N(2*I+1))->BT2{2}N(2*I+1)->[BT2{2}NI]*2,BT2{2}N
    w1 = w1.transpose(-2, -1)  # BT2{2}NI->BT2{2}IN
#     w1 = w1 * torch.rsqrt(w1.pow(2).mean(-1, keepdim=True) + 1e-6)
    w1 = F.layer_norm(w1, (N,))
    J = 'J' # if project_logits else ''
    qw, kw = torch.einsum(f'BTK{J}IN,BTK{J}MI->BK{J}NMT', w1, w2).unbind(dim=1)  # j=k=2, B2{2}NMT->[B{2}NMT]*2
    KW = torch.cat([KW, kw], dim=-1)  # B{2}NMS,B{2}NM1->B{2}NM(S+1)
    w = sw.unsqueeze(-1) + qw + KW  # {2}NM1,B{2}NM1,B{2}NMS->B{2}NMS
    dims = (0, 2, 3, 4, 1) # if project_logits else (0, 2, 3, 1)
    qdd, kdd = F.tanh(dd).squeeze(-1).permute(dims).unbind(dim=1) # BT2{2}N1->BT2{2}N->B2{2}NT->[B{2}NT]*2
    KDD = torch.cat([KDD, kdd], dim=-1)  # B{2}NS,B{2}N1->B{2}N(S+1)
    d = qdd + KDD  # B{2}N1,B{2}NS->B{2}NS
    if project_logits:
        wl, w = w.unbind(1); dl, d = d.unbind(1)
        logits = torch.einsum('BNTS,BNMS->BMTS', logits, wl) + torch.einsum('BNTS,BNS->BNTS', logits, dl)
    probs = logits.softmax(-1)
    probs = torch.einsum('BNTS,BNMS->BMTS', probs, w) + torch.einsum('BNTS,BNS->BNTS', probs, d)
    
    o = probs @ V  # BNTS,BNSD->BNTD
    o = o.permute((0, 2, 1, 3)).view(B, T, E) @ wo  # BNTD->BTND->BT(ND)
    return K, V, KW, KDD

In [ ]:
B, T, S, N, D, I = 1, 1, 1024, 32, 128, 2; E = D * N
# w3 = torch.randn(E, E * 3).to(0)
wq = torch.randn(E, E).to(0); wk = torch.randn(E, E).to(0); wv = torch.randn(E, E).to(0); wo = torch.randn(E, E).to(0)  
K = torch.randn(B, N, S, D).to(0); V = torch.randn(B, N, S, D).to(0)
project_logits = False
if project_logits:
    sw = torch.randn(2, N, N).to(0)
    dw = torch.randn(E, 2 * 2 * N * (2 * I + 1)).to(0)
    KW = torch.randn(B, 2, N, N, S).to(0)  
    KDD = torch.randn(B, 2, N, S).to(0)
else:
    sw = torch.randn(N, N).to(0)
    dw = torch.randn(E, 2 * N * (2 * I + 1)).to(0)
    # KW = torch.randn(B, S, N, N).to(0)
    KW = torch.randn(B, N, N, S).to(0)  
    # KDD = torch.randn(B, S, N).to(0)
    KDD = torch.randn(B, N, S).to(0)

n_iters = 50
times = []
for i in range(n_iters):
    x = torch.randn(B, T, E).to(0)
    with torch.no_grad(): (K, V, KW, KDD), t = timed(lambda: attn(x, wq, wk, wv, wo, sw, dw, K, V, KW, KDD))
    times.append(t)
plt.plot(times); # print(times)
print(np.mean(np.array(times[5:])))

In [ ]:
baseline 254
post 310
post+rms 310~380
post+F.rms 310
pre+post+F.rms 314

not jit
baseline 333
pre+post+F.rms 740

In [ ]:
@torch.jit.script
def model(x, wqs, wks, wvs, wos, sws, dws, qkws, Ks, Vs, KWs, KDDs, W1s, Wgs, W2s, We, Wu):
    # B, T, S, N, D, I = 1, 1, 1024, 16, 128, 2; E = D * N; L=28 # 1.4B
    B, T, S, N, D, I = 1, 1, 1024, 32, 128, 2; E = D * N; L=32 # 6.7B
    x = We[x]
    Ks_new, Vs_new, KWs_new, KDDs_new = [], [], [], [] 
    for lidx in range(L): 
        wq, wk, wv, wo, sw, dw, qkw, K, V, KW, KDD, W1, Wg, W2 = [w[lidx] for w in 
            [wqs, wks, wvs, wos, sws, dws, qkws, Ks, Vs, KWs, KDDs, W1s, Wgs, W2s]]
        window_size = 256 if lidx % 2 == 0 else None
        if window_size is not None:
            K, V = K[:, :, -window_size+1:], V[:, :, -window_size+1:]  # BNSD->BNWD
            KW = KW[:, -window_size+1:]  # BS2NN->BW2NN
        residual = x
        x = F.layer_norm(x, (E,)) # pre layernorm
        #Attn
        q, k, v = x @ wq, x @ wk, x @ wv
        q, k, v = [a.view(B, T, N, D).permute((0, 2, 1, 3)) for a in [q, k, v]]  # BTND->BNTD
        K = torch.cat([K, k], dim=2); V = torch.cat([V, v], dim=2)  # BNSD,BN1D->BN(S+1)D
        logits = q @ K.transpose(-2, -1)  # BNTS
        
        project_logits = True # lidx % 4 == 1
        
        dw_hidden, dd = (x @ dw).split([2*2*N*(2*I), 2*2*N*1], -1) # BT(4K), BT4N         # K=2*N*I
        dw_hidden = F.gelu(dw_hidden) 
        dw_hidden = dw_hidden.view(dw_hidden.shape[:2]+(4,-1)) #B T (4 K) -> B T 4 K  # reshape
        dw = torch.einsum('B T C K, C K D -> B T C D', dw_hidden, qkw) # BT4K,4K(MI)->BT4(MI)
#         shape = (B,T,2,2,N,-1) # if project_logits else (B,T,2,N,-1)
#         w1, w2 = dw.view(shape).split(I,-1) # BT22N(2*I) -> 2*[BT22NI]:BT(pre/post)(q/k)NI
#         w1 = w1.transpose(-2, -1)  # BT2{2}NI->BT2{2}IN
        shape = (B,T,2,2,-1,N)
        w1, w2 = dw.view(shape).split(I,-2)
        
    #     w1 = w1 * torch.rsqrt(w1.pow(2).mean(-1, keepdim=True) + 1e-6)
        w1 = F.layer_norm(w1, (N,))
        J = 'J' # if project_logits else ''

#         qkw = torch.einsum(f'BTK{J}IN,BTK{J}MI->BTK{J}NM', w1, w2)  # j=k=2, BT2{2}NM
#         qkdd = F.tanh(dd).squeeze(-1).view(shape[:-1]) # BT2{2}N1->BT2{2}N
        qkw = torch.einsum(f'BTK{J}IN,BTK{J}IM->BTK{J}NM', w1, w2)  # j=k=2, BT2{2}NM
        qkdd = F.tanh(dd).squeeze(-1).view(shape[:-2] + (N,)) # BT2{2}N1->BT2{2}N
        
        qkw = qkw + torch.diag_embed(qkdd) # BT2{2}NN,->BT2{2}NN
        qw, kw = qkw.unbind(2)  # B2{2}NNT-> [B{2}NMT]*2
        KW = torch.cat([KW, kw], dim=1)  # BS{2}NM,B1{2}NM->B(S+1){2}NM
        w = sw  + qw + KW  #
        
        # qw, kw = torch.einsum(f'BTK{J}IN,BTK{J}MI->BK{J}NMT', w1, w2).unbind(dim=1)  # j=k=2, B2{2}NMT->[B{2}NMT]*2
        # KW = torch.cat([KW, kw], dim=-1)  # B{2}NMS,B{2}NM1->B{2}NM(S+1)
        # # print('sw qw, kw shape', sw.shape, qw.shape, KW.shape)
        # w = sw.unsqueeze(-1) + qw + KW  # {2}NM1,B{2}NM1,B{2}NMS->B{2}NMS
        # dims = (0, 2, 3, 4, 1) # if project_logits else (0, 2, 3, 1)
        # dd = F.tanh(dd); dd = dd.view(shape[:-1])
        # qdd, kdd = dd.squeeze(-1).permute(dims).unbind(dim=1) # BT2{2}N1->BT2{2}N->B2{2}NT->[B{2}NT]*2
        # KDD = torch.cat([KDD, kdd], dim=-1)  # B{2}NS,B{2}N1->B{2}N(S+1)
        # d = qdd + KDD  # B{2}N1,B{2}NS->B{2}NS
        
        wl, w = w.unbind(2) # dl, d = d.unbind(1)
        if project_logits:
            logits = torch.einsum('BNTS,BSNM->BMTS', logits, wl) #+ torch.einsum('BNTS,BNS->BNTS', logits, dl)
        probs = logits.softmax(-1)
        probs = torch.einsum('BNTS,BSNM->BMTS', probs, w) #+ torch.einsum('BNTS,BNS->BNTS', probs, d)
        
        o = probs @ V  # BNTS,BNSD->BNTD
        o = o.permute((0, 2, 1, 3)).view(B, T, E) @ wo  # BNTD->BTND->BT(ND)
        Ks_new.append(K);Vs_new.append(V);KWs_new.append(KW)#; KDDs_new.append(KDD) 
        x = residual + o
        #MLP
        parallel_mlp = False
        xn = F.layer_norm(residual if parallel_mlp else x, (E,))
        mlp_out = ((xn @ W1) * F.silu(xn @ Wg)) @ W2
        x = x + mlp_out
    x = F.layer_norm(x, (E,))
    x = (x @ Wu).softmax(dim=-1)
    x = torch.argmax(x, dim=-1) # greedy generation
    return x, Ks_new, Vs_new, KWs_new, KDDs_new

In [ ]:
def make_causal_mask(shape, dtype):
    bsz, tgt_len = shape
    mask = torch.full((tgt_len, tgt_len), torch.finfo(dtype).min * 0.5)
    mask_cond = torch.arange(mask.size(-1))
    mask.masked_fill_(mask_cond < (mask_cond + 1).view(mask.size(-1), 1), 0)
    return mask
        
def _cross_head_proj(inputs, sw, qw1, qw2, kw1, kw2, qdd, kdd, loop_over_dynamic_hd=False):
    out = inputs + torch.einsum('BNTS,NM->BMTS', inputs, sw)
    if loop_over_dynamic_hd:
        for i in range(qw1.shape[-2]):
            qhidden = torch.einsum('BNTS,BTN->BTS', inputs, qw1[..., i, :])
            qout = torch.einsum('BTS,BTN->BNTS', qhidden, qw2[..., i, :]); out = out + qout
            khidden = torch.einsum('BNTS,BSN->BTS', inputs, kw1[..., i, :])
            kout = torch.einsum('BTS,BSN->BNTS', khidden, kw2[..., i, :]); out = out + kout
    else:
        qhidden = torch.einsum('BNTS,BTIN->BITS', inputs, qw1)
        qout = torch.einsum('BITS,BTIN->BNTS', qhidden, qw2); out = out + qout
        khidden = torch.einsum('BNTS,BSIN->BITS', inputs, kw1)
        kout = torch.einsum('BITS,BSIN->BNTS', khidden, kw2); out = out + kout
    qdout = torch.einsum('BNTS,BTN->BNTS', inputs, qdd); out = out + qdout
    kdout = torch.einsum('BNTS,BSN->BNTS', inputs, kdd); out = out + kdout
    return out
    
def _atten_context(query, key, value, atten_mask, pre_proj_dw_args, post_proj_dw_args):
    logits = query @ key.transpose(-2, -1)
    if pre_proj_dw_args is not None: logits = _cross_head_proj(logits, *pre_proj_dw_args)
    logits = logits + atten_mask
    probs = logits.softmax(-1)
    if post_proj_dw_args is not None: probs = _cross_head_proj(probs, *post_proj_dw_args)
    o = probs @ value  # BNTS,BNSD->BNTD
    return o

In [ ]:
@torch.compile
def model(x, labels, atten_mask, wqs, wks, wvs, wos, sws, dws, qkws, W1s, Wgs, W2s, We, Wu):  # Ks, Vs, KWs, KDDs, 
    # B, T, S, N, D, I = 1, 1, 1024, 16, 128, 2; E = D * N; L=28 # 1.4B
    B, T, S, N, D, I = 1, 2048 // 2, 2048 // 2, 32, 128, 2; E = D * N; L=32 # 6.7B
    q_chunk_size = T # 128
    x = We[x]
    for lidx in range(L): 
        wq, wk, wv, wo, sw, dw, qkw, W1, Wg, W2 = [w[lidx] for w in   # K, V, KW, KDD, 
            [wqs, wks, wvs, wos, sws, dws, qkws, W1s, Wgs, W2s]]  # Ks, Vs, KWs, KDDs, 
        window_size = 256 if lidx % 2 == 0 else S
        residual = x
        x = F.layer_norm(x, (E,)) # pre layernorm
        #Attn
        q, k, v = x @ wq, x @ wk, x @ wv
        q, k, v = [a.view(B, T, N, D).permute((0, 2, 1, 3)) for a in [q, k, v]]  # BTND->BNTD
        
        project_logits = True # lidx % 4 == 1
        project_probs = True
        if project_probs:
            dw_hidden, dd = (x @ dw).split([2*2*N*(2*I), 2*2*N*1], -1) # BT(4K), BT4N         # K=2*N*I
            dw_hidden = F.gelu(dw_hidden) 
            dw_hidden = dw_hidden.view(dw_hidden.shape[:2]+(4,-1)) #B T (4 K) -> B T 4 K  # reshape
            dw = torch.einsum('B T C K, C K D -> B T C D', dw_hidden, qkw) # BT4K,4K(MI)->BT4(MI)
            shape = (B,T,2*2,-1,N)# if project_logits else (B,T,2,N,-1)  # BT(pre/post)(q/k)IN
            w1, w2 = dw.view(shape).split(I,-2)
            w1 = F.layer_norm(w1, (N,)) # w1 = w1 * torch.rsqrt(w1.pow(2).mean(-1, keepdim=True) + 1e-6)
            J = 'J' # if project_logits else ''
    
            pre_sw, post_sw = sw.unbind(0)
            pre_qw1, pre_kw1, post_qw1, post_kw1 = w1.unbind(2)  # BT(2{*2})IN->[BTIN]*4
            pre_qw2, pre_kw2, post_qw2, post_kw2 = w2.unbind(2)
            qkdd = F.tanh(dd).squeeze(-1).view(shape[:-2] + (N,)) # BT(2{*2})N1->BT(2{*2})N
            pre_qdd, pre_kdd, post_qdd, post_kdd = qkdd.unbind(2)  # BT(2{*2})N->[BTN]*4
        
        o = torch.zeros(B, N, T, D).to(q.device, dtype=q.dtype)
        for i in range(T // q_chunk_size):
            start, stop = i * q_chunk_size, (i + 1) * q_chunk_size
            kv_start = max(0, stop - q_chunk_size - window_size)
            _q = q[:, :, start : stop, :]
            _k, _v = k[:, :, kv_start : stop, :], v[:, :, kv_start : stop, :]
            _atten_mask = atten_mask[:, :, start : stop, kv_start : stop]
            def slice_dw(sw, qw1, qw2, kw1, kw2, qdd, kdd):
                return (sw,
                        qw1[:, start : stop] if qw1 is not None else None,
                        qw2[:, start : stop] if qw2 is not None else None,
                        kw1[:, kv_start : stop] if kw1 is not None else None,
                        kw2[:, kv_start : stop] if kw2 is not None else None,
                        qdd[:, start : stop] if qdd is not None else None,
                        kdd[:, kv_start : stop] if kdd is not None else None)
            _pre_proj_dw_args = slice_dw(pre_sw, pre_qw1, pre_qw2, pre_kw1, pre_kw2, pre_qdd, pre_kdd) if project_logits else None
            _post_proj_dw_args = slice_dw(post_sw, post_qw1, post_qw2, post_kw1, post_kw2, post_qdd, post_kdd) if project_probs else None
            _o = _atten_context(_q, _k, _v, _atten_mask, _pre_proj_dw_args, _post_proj_dw_args)
            o[:,:,start:stop] = _o
            
        o = o.permute((0, 2, 1, 3)).reshape(B, T, E) @ wo  # BNTD->BTND->BT(ND)
        x = residual + o
        #MLP
        parallel_mlp = False
        xn = F.layer_norm(residual if parallel_mlp else x, (E,))
        mlp_out = ((xn @ W1) * F.silu(xn @ Wg)) @ W2
        x = x + mlp_out
    x = F.layer_norm(x, (E,))
    logits = x.float() @ Wu
    loss = F.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1))
    print('loss =', loss.item())
    return loss

In [ ]:
device = 1; dtype = torch.float16
B, T, S, N, D, I = 1, 2048 // 2, 2048 // 2, 32, 128, 2; E = D * N; L=32 # 6.7B
VS= 50257
IE = 11008 # int(8/3 * E)  # XD

In [ ]:
# w3 = torch.randn(E, E * 3).to(0)
wqs = (torch.randn(L, E, E) * 1e-5).to(device, dtype).requires_grad_() #* 1e-5
wks = (torch.randn(L, E, E)* 1e-5).to(device, dtype).requires_grad_() #* 1e-5
wvs = (torch.randn(L, E, E)* 1e-5).to(device, dtype).requires_grad_() #* 1e-5
wos = (torch.randn(L, E, E)* 1e-5).to(device, dtype).requires_grad_() #* 1e-5
# Ks = torch.randn(L, B, N, S, D).to(device, dtype)
# Vs = torch.randn(L, B, N, S, D).to(device, dtype)

sws = (torch.randn(L, 2, N, N) * 1e-5).to(device, dtype).requires_grad_() #* 1e-5
dws = (torch.randn(L, E, 2 * 2 * N * (2 * I + 1))* 1e-5).to(device, dtype).requires_grad_() #* 1e-5
qkws = (torch.randn(L, 4, 2*N*I, N*I*2)* 1e-5).to(device, dtype).requires_grad_() #* 1e-5 # new
# KWs = torch.randn(L, B, S, 2, N, N).to(device, dtype)  # permutated
# KDDs = torch.randn(L, B, S, 2, N).to(device, dtype) # permutated

W1s = (torch.randn(L, E, IE)* 1e-5).to(device, dtype).requires_grad_() #* 1e-5
Wgs = (torch.randn(L, E, IE)* 1e-5).to(device, dtype).requires_grad_() #* 1e-5
W2s = (torch.randn(L, IE, E)* 1e-5).to(device, dtype).requires_grad_() #* 1e-5

We = (torch.randn(VS, E)* 1e-5).to(device, dtype).requires_grad_() #* 1e-5
Wu = (torch.randn(E, VS)* 1e-5).to(device, torch.float32).requires_grad_() #* 1e-5

In [ ]:
def train_step(x, labels, mask, wqs, wks, wvs, wos, sws, dws, qkws, W1s, Wgs, W2s, We, Wu):
    loss = model(x, labels, mask, wqs, wks, wvs, wos, sws, dws, qkws, W1s, Wgs, W2s, We, Wu)  # Ks, Vs, KWs, KDDs, 
    loss.backward()
    return loss

In [ ]:
n_iters = 10
times = []
mask = make_causal_mask((B, T), dtype=dtype)[None, None, :, :].to(device, dtype=dtype)

for i in range(n_iters):
    x = torch.randint(VS, (B,T)).to(device)
    labels = torch.randint(VS, (B,T)).to(device)
    _, t = timed(lambda: train_step(x, labels, mask, wqs, wks, wvs, wos, sws, dws, qkws, W1s, Wgs, W2s, We, Wu))
    # print(i, Ks[0].shape)
    times.append(t)
plt.plot(times); # print(times)
print(np.mean(np.array(times[3:])))

In [ ]:
T = 1024
baseline
0.169
taling
0.652

In [ ]:
baseline
nocompile 0.704693830217634
compile nochunk 0.2715109645298549

talking
nocompile 1.5 / 3.560845179966518
compile nochunk 0.271770769391741 compile 10x faster
compile chunk128 0.2717097691127232

In [ ]:
n_iters = 50
times = []
x = torch.randint(VS, (B,T)).to(device)
labels = torch.randint(VS, (B,T)).to(device)
mask = make_causal_mask(x, dtype=dtype)
for i in range(n_iters):
    # x = torch.randn(B, T, E).to(0)
    with torch.no_grad():
        (x, Ks_new, Vs_new, KWs_new, KDDs_new), t = timed(lambda: model(
            x, wqs, wks, wvs, wos, sws, dws, qkws, Ks, Vs, KWs, KDDs, W1s, Wgs, W2s,We,Wu))
    # print(i, Ks[0].shape)
    times.append(t)
plt.plot(times); # print(times)
print(np.mean(np.array(times[10:])))

In [ ]:
# fp16
baseline 0.0133
baseline win256 0.0124
whole 0.0160
1to4 0.0156
onlyprobs 0.0156
whole win256 0.0161?
whole win256+1to4 0.0152

# fp32
baseline 0.0272
baseline win256 0.0262
baseline paramlp 0.0270
whole 0.0306
1to4 0.0303
onlyprobs 0.0301
whole win256 0.0292
whole win256+1to4 0.0289
whole paramlp 0.03034